# eCommerce Transactions Dataset

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import cdist
from datetime import datetime

In [2]:
def load_data():
    customers_df = pd.read_csv('Customers.csv')
    products_df = pd.read_csv('Products.csv')
    transactions_df = pd.read_csv('Transactions.csv')
    
    # Convert dates to datetime
    customers_df['SignupDate'] = pd.to_datetime(customers_df['SignupDate'])
    transactions_df['TransactionDate'] = pd.to_datetime(transactions_df['TransactionDate'])
    
    return customers_df, products_df, transactions_df

### Task 1: Exploratory Data Analysis and Business Insights

In [3]:
def perform_eda(customers_df, products_df, transactions_df):
    trans_with_customer = transactions_df.merge(customers_df, on='CustomerID')
    trans_with_product = trans_with_customer.merge(products_df, on='ProductID')
    
    total_revenue = trans_with_product['TotalValue'].sum()
    avg_order_value = trans_with_product.groupby('TransactionID')['TotalValue'].sum().mean()
    customers_by_region = customers_df['Region'].value_counts()
    sales_by_category = trans_with_product.groupby('Category')['TotalValue'].sum()
    
    trans_with_product['Month'] = trans_with_product['TransactionDate'].dt.strftime('%Y-%m')
    monthly_sales = trans_with_product.groupby('Month')['TotalValue'].sum()
    
    customer_frequency = trans_with_product.groupby('CustomerID').size()
    
    return {
        'total_revenue': total_revenue,
        'avg_order_value': avg_order_value,
        'customers_by_region': customers_by_region,
        'sales_by_category': sales_by_category,
        'monthly_sales': monthly_sales,
        'customer_frequency': customer_frequency
    }

### Task 2: Lookalike Model

In [4]:
def create_customer_features(customers_df, transactions_df, products_df):
    customer_features = transactions_df.groupby('CustomerID').agg({
        'TotalValue': ['sum', 'mean', 'count'],
        'Quantity': ['sum', 'mean']
    }).reset_index()
    
    customer_features.columns = ['CustomerID', 'total_spend', 'avg_spend', 'transaction_count', 
                               'total_quantity', 'avg_quantity']
    
    trans_with_prod = transactions_df.merge(products_df[['ProductID', 'Category']], on='ProductID')
    category_pivot = pd.pivot_table(trans_with_prod, 
                                  values='TotalValue', 
                                  index='CustomerID',
                                  columns='Category',
                                  aggfunc='sum',
                                  fill_value=0)
    
    customer_features = customer_features.merge(category_pivot, on='CustomerID')
    customer_features = customer_features.merge(customers_df[['CustomerID', 'Region']], on='CustomerID')
    
    customer_features = pd.get_dummies(customer_features, columns=['Region'])
    
    return customer_features

In [5]:
def find_lookalikes(customer_features, target_customer_id, n_recommendations=3):
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(customer_features.drop('CustomerID', axis=1))
    
    target_idx = customer_features[customer_features['CustomerID'] == target_customer_id].index[0]
    target_features = features_scaled[target_idx].reshape(1, -1)
    
    similarities = 1 - cdist(target_features, features_scaled, metric='cosine')[0]
    
    similar_indices = np.argsort(similarities)[::-1][1:n_recommendations+1]
    similar_customers = customer_features.iloc[similar_indices]
    similarity_scores = similarities[similar_indices]

    return list(zip(similar_customers['CustomerID'], similarity_scores))

### Task 3: Customer Segmentation

In [6]:
def perform_clustering(customer_features, n_clusters=5):
    # Prepare data for clustering
    features_scaled = StandardScaler().fit_transform(customer_features.drop('CustomerID', axis=1))
    
    # Perform K-means clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(features_scaled)
    
    # Calculate Davies-Bouldin Index
    db_index = davies_bouldin_score(features_scaled, clusters)
    
    # Add cluster labels to customer features
    customer_features['Cluster'] = clusters
    
    return customer_features, db_index

In [8]:
customers_df, products_df, transactions_df = load_data()

eda_results = perform_eda(customers_df, products_df, transactions_df)

customer_features = create_customer_features(customers_df, transactions_df, products_df)

lookalike_results = {}
for cust_id in customers_df['CustomerID'].iloc[:20]:
    lookalikes = find_lookalikes(customer_features, cust_id)
    lookalike_results[cust_id] = lookalikes

lookalike_df = pd.DataFrame([(k, v[0][0], v[0][1], v[1][0], v[1][1], v[2][0], v[2][1]) 
                            for k, v in lookalike_results.items()],
                            columns=['CustomerID', 'Similar1', 'Score1', 
                                    'Similar2', 'Score2', 'Similar3', 'Score3'])
lookalike_df.to_csv('Lookalike.csv', index=False)

clustered_customers, db_index = perform_clustering(customer_features)

plt.figure(figsize=(10, 6))
sns.scatterplot(data=clustered_customers, 
                x='total_spend', 
                y='transaction_count',
                hue='Cluster',
                palette='deep')
plt.title('Customer Segments by Total Spend and Transaction Count')
plt.savefig('cluster_visualization.png')
plt.close()

eda_results, db_index, clustered_customers

/Users/varshit_madi/Library/Python/3.9/lib/python/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


({'total_revenue': 689995.56,
  'avg_order_value': 689.9955600000001,
  'customers_by_region': South America    59
  Europe           50
  North America    46
  Asia             45
  Name: Region, dtype: int64,
  'sales_by_category': Category
  Books          192147.47
  Clothing       166170.66
  Electronics    180783.50
  Home Decor     150893.93
  Name: TotalValue, dtype: float64,
  'monthly_sales': Month
  2023-12     3769.52
  2024-01    66376.39
  2024-02    51459.27
  2024-03    47828.73
  2024-04    57519.06
  2024-05    64527.74
  2024-06    48771.18
  2024-07    71366.39
  2024-08    63436.74
  2024-09    70603.75
  2024-10    47063.22
  2024-11    38224.37
  2024-12    59049.20
  Name: TotalValue, dtype: float64,
  'customer_frequency': CustomerID
  C0001    5
  C0002    4
  C0003    4
  C0004    8
  C0005    3
          ..
  C0196    4
  C0197    3
  C0198    2
  C0199    4
  C0200    5
  Length: 199, dtype: int64},
 1.46128906179447,
     CustomerID  total_spend    avg_spe